# 🧠 Cortex-8: Transfer Learning & Adaptation

## 🦎 La Capacidad de Adaptación
La verdadera inteligencia no es solo memorizar Shakespeare; es la capacidad de aprender nuevas habilidades.
En este experimento:
1.  **El Poeta**: Entrenaremos un modelo experto en Literatura (Shakespeare).
2.  **El Choque Cultural**: Le pediremos que resuelva Matemáticas (sin haberlas visto nunca).
3.  **La Adaptación**: Lo "re-entrenaremos" (Fine-Tuning) con un nuevo dataset de Lógica/Matemáticas.
4.  **El Resultado**: Veremos si un Poeta puede convertirse en Matemático.

---

In [ ]:
# 1. Setup & Librerías
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import requests
import numpy as np
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Cortex-8 Engine: {device.upper()}")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# 2. Arquitectura Híbrida (La misma de siempre)
class MambaBlock(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.in_proj = nn.Linear(d_model, d_model * 2)
        self.out_proj = nn.Linear(d_model, d_model)
        self.conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
    def forward(self, x):
        B, L, D = x.shape
        x_and_res = self.in_proj(x)
        x_val, res = x_and_res.chunk(2, dim=-1)
        x_val = x_val.transpose(1, 2)
        x_val = self.conv(x_val)
        x_val = x_val.transpose(1, 2)
        x_val = F.silu(x_val)
        return self.out_proj(x_val * F.sigmoid(res))

class CortexOrganism(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embedding = nn.Embedding(256, config['d_model'])
        self.layers = nn.ModuleList()
        for i in range(config['n_layers']): 
            if i % 2 == 0: self.layers.append(MambaBlock(config['d_model']))
            else: self.layers.append(nn.TransformerEncoderLayer(
                d_model=config['d_model'], nhead=config['n_heads'], 
                dim_feedforward=4*config['d_model'], batch_first=True, dropout=0.1
            ))
        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], 256)

    def forward(self, idx, targets=None):
        x = self.embedding(idx)
        for layer in self.layers: x = layer(x)
        x = self.ln_f(x)
        logits = self.head(x)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

def generate(model, prompt, max_len=50):
    model.eval()
    idx = torch.tensor([ord(c) for c in prompt], dtype=torch.long).unsqueeze(0).to(device)
    for _ in range(max_len):
        with torch.no_grad():
            logits, _ = model(idx)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            next_token = torch.multinomial(probs, 1)
            idx = torch.cat((idx, next_token), dim=1)
    return "".join([chr(i) for i in idx[0].tolist()])

### 3. Fase A: Entrenar al Poeta (Shakespeare)
Cargamos el dataset literario y entrenamos al modelo base.

In [ ]:
# Dataset A: Literatura
shakespeare_text = requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt").text
data_A = torch.tensor([ord(c) for c in shakespeare_text], dtype=torch.long)

def get_batch_A():
    ix = torch.randint(len(data_A) - 64, (32,))
    x = torch.stack([data_A[i:i+64] for i in ix]).to(device)
    y = torch.stack([data_A[i+1:i+65] for i in ix]).to(device)
    return x, y

print("🎭 Entrenando al Poeta...")
config = {'n_layers': 4, 'd_model': 256, 'n_heads': 4}
model = CortexOrganism(config).to(device)
optim = torch.optim.AdamW(model.parameters(), lr=1e-3)

losses_A = []
for i in range(500): # Entrenamiento medio
    xb, yb = get_batch_A()
    _, loss = model(xb, yb)
    optim.zero_grad()
    loss.backward()
    optim.step()
    losses_A.append(loss.item())
    if i % 100 == 0: print(f"   Iter {i}: Loss {loss.item():.4f}")

print(f"\n📜 El Poeta dice: \"{generate(model, 'The king ', 50)}\"")

### 4. Fase B: El Nuevo Mundo (Matemáticas)
Generamos un dataset sintético de aritmética simple. Esto es un lenguaje totalmente distinto al inglés isabelino.

In [ ]:
def generate_math_dataset(size=10000):
    text = ""
    for _ in range(size):
        a = random.randint(1, 99)
        b = random.randint(1, 99)
        op = random.choice(['+', '-', '*'])
        if op == '+': res = a + b
        elif op == '-': res = a - b
        else: res = a * b
        text += f"Q: {a}{op}{b}=? A: {res}\n"
    return text

math_text = generate_math_dataset()
print(f"🧮 Dataset Matemático Generado ({len(math_text)} chars)")
print("--- Muestra ---")
print(math_text[:100])

data_B = torch.tensor([ord(c) for c in math_text], dtype=torch.long)

def get_batch_B():
    ix = torch.randint(len(data_B) - 64, (32,))
    x = torch.stack([data_B[i:i+64] for i in ix]).to(device)
    y = torch.stack([data_B[i+1:i+65] for i in ix]).to(device)
    return x, y

### 5. Prueba Zero-Shot (Antes de Re-entrenar)
¿Puede el Poeta resolver matemáticas sin estudiar? (Spoiler: No)

In [ ]:
print("🧪 Prueba Zero-Shot (Poeta intentando Matemáticas):")
prompt = "Q: 10+10=? A: "
response = generate(model, prompt, 10)
print(f"   Entrada: '{prompt}'")
print(f"   Salida:  '{response}' (Probablemente alucinación)")

### 6. Fase C: Fine-Tuning (La Adaptación)
Tomamos al Poeta y le damos clases intensivas de matemáticas.

In [ ]:
print("🎓 Re-entrenando (Fine-Tuning) en Matemáticas...")
# Bajamos un poco el Learning Rate para no destruir el cerebro previo demasiado rápido
optim = torch.optim.AdamW(model.parameters(), lr=5e-4)

losses_B = []
for i in range(300): # Entrenamiento corto de adaptación
    xb, yb = get_batch_B()
    _, loss = model(xb, yb)
    optim.zero_grad()
    loss.backward()
    optim.step()
    losses_B.append(loss.item())
    if i % 50 == 0: print(f"   Iter {i}: Loss {loss.item():.4f}")

print("✅ Adaptación Completada.")

### 7. Verificación Final
¿Puede ahora resolver problemas?

In [ ]:
print("🧪 Prueba Final (Poeta convertido en Matemático):")
test_prompts = [
    "Q: 5+5=? A: ",
    "Q: 10-2=? A: ",
    "Q: 2*3=? A: "
]

for p in test_prompts:
    res = generate(model, p, 5).split('\n')[0] # Tomamos solo la primera línea
    print(f"   {p}{res}")
    
# Visualizar el cambio de mentalidad
plt.figure(figsize=(10, 5))
plt.plot(losses_A, label='Fase A: Literatura')
plt.plot(range(500, 800), losses_B, label='Fase B: Matemáticas')
plt.axvline(x=500, color='r', linestyle='--', label='Cambio de Dominio')
plt.title("Evolución del Aprendizaje: De Poeta a Matemático")
plt.legend()
plt.show()